In [1]:
import sys, pandas as pd; [sys.path.append('../' * i) for i in range(1,7)]
import pprint 
import json 
from dotenv import load_dotenv
from pydantic import BaseModel, Field, create_model
from pydantic_ai import format_as_xml
from securitygpt.agents.core.base_prompt import BasePromptStructure
from securitygpt.agents.core.base_agent import AgentCreationPrompt
from securitygpt.agents.core.base_agent import BaseAgents 


from securitygpt.agents.services.llms.chat.chat_completion_v2 import ChatCompletion as chat_complete

from securitygpt.agents.services.research.tools.serp_tools import search_news, search_organic, get_objective_to_search_queries
from securitygpt.agents.experts.search.utils.llm_search_utils import get_objective_to_search_queries
# __file__ is not defined in Jupyter notebooks, so we'll just print the env loading
env_path = load_dotenv()
print("Loading .env file from:", env_path)


Loading .env file from: True


In [2]:
topic = "research on the topic: latest threats and campaigns in the last week"
function_objects = [search_news, search_organic, get_objective_to_search_queries]   


In [3]:

function_names_and_docs = "\n".join([f"{func.__name__}: {func.__doc__ or 'No documentation'}" for func in function_objects])


# Usage (updated)
rules = f""" 
"Each agent must have a simple, single-focused task and primary tool execution responsibility",
"Take user query, chunk it into the smallest sub tasks possible, and assign each sub task to an agent",
"Manager agent must orchestrate workflow and coordinate between agents",
"Evaluator agent must assess quality and completeness of outputs", 
"Worker agents must execute their assigned tools and tasks effectively",
"All agents must follow a clear step-by-step plan",
"Each step must have a defined owner and success criteria",
"Output must be structured with clear sections and subsections",
"Agents must coordinate through the manager for handoffs",
"The agents must be created with the following functions: {function_names_and_docs}",
"You need to consider the functions and arguments when creating the agents and assign the functions along with specific arguments to the agents"

""" 

# New usage style
AgentCreationPrompt = AgentCreationPrompt.build(rules=rules, job=topic)


In [4]:

prompt = json.loads(chat_complete().create(user_input=topic, output_format=AgentCreationPrompt).result)
print (prompt)
user_prompt_xml = format_as_xml(prompt, root_tag='agent_creation_prompt')
pprint.pprint (user_prompt_xml)

INFO:securitygpt.agents.services.llms.chat.chat_completion_v2:Using model 'gemini-2.0-flash-lite' from 'google'
INFO:securitygpt.agents.services.llms.chat.providers._google:Configuring structured output for AgentCreationPrompt
INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-lite:generateContent "HTTP/1.1 200 OK"
INFO:google_genai.models:AFC remote call 1 is done.


{'role': 'research assistant', 'job': 'research on the topic: latest threats and campaigns in the last week', 'context': 'Each agent must have a simple, single-focused task and primary tool execution responsibility. Take user query, chunk it into the smallest sub tasks possible, and assign each sub task to an agent. Manager agent must orchestrate workflow and coordinate between agents. Evaluator agent must assess quality and completeness of outputs. Worker agents must execute their assigned tools and tasks effectively. All agents must follow a clear step-by-step plan. Each step must have a defined owner and success criteria. Output must be structured with clear sections and subsections. Agents must coordinate through the manager for handoffs. The agents must be created with the following functions: search_news: this tool takes a query, limit, and num_weeks and returns a list of news article links that will have news articles from the last num_weeks weeks related to the query. Args: que

In [5]:
json.loads(chat_complete().create(user_input=str(user_prompt_xml), output_format=BaseAgents).result)


INFO:securitygpt.agents.services.llms.chat.chat_completion_v2:Using model 'gemini-2.0-flash-lite' from 'google'
INFO:securitygpt.agents.services.llms.chat.providers._google:Configuring structured output for BaseAgents
INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-lite:generateContent "HTTP/1.1 200 OK"
INFO:google_genai.models:AFC remote call 1 is done.


{'agents': [{'name': 'Query Generation Agent',
   'instructions': 'Generate a list of search queries related to the research objective. These queries should be specific and relevant to uncover the latest threats and campaigns. Prioritize queries that are most likely to yield recent information.',
   'functions': ['get_objective_to_search_queries'],
   'reason': 'This agent is crucial for initiating the research process by identifying the most effective search terms to gather relevant information about the latest threats and campaigns.'},
  {'name': 'News Search Agent',
   'instructions': 'Search for news articles related to the provided queries, focusing on results from the last week. This agent will utilize the `search_news` tool to find the latest news reports on threats and campaigns.',
   'functions': ['search_news'],
   'reason': 'This agent is responsible for gathering timely information from news sources, which are essential for understanding the most recent threat landscape.'},